In [1]:
from __future__ import division

from models import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *

import os
import sys
import time
import datetime
import argparse

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

In [2]:
cuda = torch.cuda.is_available() and opt.use_cuda
cuda

False

In [3]:
os.makedirs("checkpoints", exist_ok=True)

In [4]:
classes = load_classes("config/coco.names")
print(classes)
print(len(classes))

['Nenhum', 'Bulbasaur', 'Ivysaur', 'Venusaur', 'Charmander', 'Charmeleon', 'Charizard', 'Squirtle', 'Wartorlte', 'Blastoise', 'Nenhum2', 'Nenhum3', 'Nenhum4', 'Nenhum5', 'Nenhum6', 'Nenhum7', 'Nenhum8', 'Nenhum9', 'Nenhum10', 'Nenhum11', 'Nenhum12', 'Nenhum13', 'Nenhum14', 'Nenhum15', 'Nenhum16', 'Pikachu']
26


In [5]:
data_config = parse_data_config("config/coco.data")
data_config

{'gpus': '0,1,2,3',
 'num_workers': '10',
 'classes': '26',
 'train': 'data/pokemons/train.txt',
 'valid': 'data/pokemons/val.txt',
 'names': 'config/coco.names',
 'backup': 'backup/'}

In [6]:
train_path = data_config["train"]
train_path

'data/pokemons/train.txt'

In [7]:
# Hiperparâmetros
hyperparams = parse_model_config("config/yolov3.cfg")[0]
learning_rate = float(hyperparams["learning_rate"])
momentum = float(hyperparams["momentum"])
decay = float(hyperparams["decay"])
burn_in = int(hyperparams["burn_in"])

print("learning_rate: %lf | momentum: %lf | decay: %lf | burn_in: %lf" % (learning_rate, 
                                                                         momentum,
                                                                         decay,
                                                                         burn_in))

learning_rate: 0.002000 | momentum: 0.900000 | decay: 0.000500 | burn_in: 1000.000000


In [8]:
model = Darknet("config/yolov3.cfg")
model.load_weights("config/yolov3.weights")

/home/wesley/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [9]:
if cuda:
    model = model.cuda()

In [10]:
model.train()

Darknet(
  (module_list): ModuleList(
    (0): Sequential(
      (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_0): LeakyReLU(negative_slope=0.1)
    )
    (1): Sequential(
      (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_1): LeakyReLU(negative_slope=0.1)
    )
    (2): Sequential(
      (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (batch_norm_2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_2): LeakyReLU(negative_slope=0.1)
    )
    (3): Sequential(
      (conv_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_3): BatchNorm2d(64, eps=1e-05, momen

In [11]:
dataloader = torch.utils.data.DataLoader(
    ListDataset(train_path), batch_size=16, shuffle=False, num_workers=2
)

In [12]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [13]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))

In [14]:
output_layer_indices = [105, 106, 93, 94, 82, 81]
freeze_layer_indices = [x for x in range(len(model.module_list)) if
                        (x not in output_layer_indices) and
                        (x - 1 not in output_layer_indices)]
for idx in freeze_layer_indices:
    for parameter in model.module_list[idx].parameters():
        parameter.requires_grad_(False)

In [15]:
#for idx, module in enumerate(model.module_list):
#    print(module)
#    print(idx)

#output_layer_indices = [82, 94, 106]

In [ ]:
for epoch in range(2):
    for batch_i, (_, imgs, targets) in enumerate(dataloader):
        imgs = Variable(imgs.type(Tensor))
        targets = Variable(targets.type(Tensor), requires_grad=False)

        optimizer.zero_grad()

        loss = model(imgs, targets)

        loss.backward()
        optimizer.step()

        print(
            "[Epoch %d/%d, Batch %d/%d] [ total %f, recall: %.5f, precision: %.5f, accuracy: %.5f]"
            % (
                epoch,
                2,
                batch_i,
                len(dataloader),
                loss.item(),
                model.losses["recall"],
                model.losses["precision"],
                model.losses["accuracy"]
            )
        )

        model.seen += imgs.size(0)

    if epoch % 10 == 0:
        model.save_weights("%s/%d.weights" % ("checkpoints", epoch))

/home/wesley/Documentos/Turing/CV/pytorch_custom_yolo_training/models.py:197: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:20.)
  loss_x = self.mse_loss(x[mask], tx[mask])
/home/wesley/Documentos/Turing/CV/pytorch_custom_yolo_training/models.py:198: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:20.)
  loss_y = self.mse_loss(y[mask], ty[mask])
/home/wesley/Documentos/Turing/CV/pytorch_custom_yolo_training/models.py:199: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:20.)
  loss_w = self.mse_loss(w[mask], tw[mask])
/home/wesley/Documentos/Turing/CV/pytorch_custom_yolo_training/models.py:200: UserWarning: ind

[Epoch 0/20, Batch 0/138] [ total 69.331268, recall: 0.00000, precision: 0.00000, accuracy: 0.00000]
[Epoch 0/20, Batch 1/138] [ total 60.394836, recall: 0.00000, precision: 0.00000, accuracy: 0.00000]
[Epoch 0/20, Batch 2/138] [ total 42.431633, recall: 0.06250, precision: 0.00008, accuracy: 0.00000]
[Epoch 0/20, Batch 3/138] [ total 40.159950, recall: 0.00000, precision: 0.00000, accuracy: 0.00000]
[Epoch 0/20, Batch 4/138] [ total 31.421156, recall: 0.00000, precision: 0.00000, accuracy: 0.00000]
[Epoch 0/20, Batch 5/138] [ total 36.636230, recall: 0.00000, precision: 0.00000, accuracy: 0.00000]
[Epoch 0/20, Batch 6/138] [ total 28.721758, recall: 0.00000, precision: 0.00000, accuracy: 0.00000]
[Epoch 0/20, Batch 7/138] [ total 28.884853, recall: 0.00000, precision: 0.00000, accuracy: 0.00000]
[Epoch 0/20, Batch 8/138] [ total 28.313650, recall: 0.00000, precision: 0.00000, accuracy: 0.00000]
[Epoch 0/20, Batch 9/138] [ total 19.866196, recall: 0.00000, precision: 0.00000, accuracy: